In [ ]:
p_storage_dir = "../../storage"

In [96]:
import pandas as pd
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Docentes

Para ter um dataset dos docentes do IFG com a maior quantidade de dados sobre eles é necessário cruzar as informações do dataset de servidores com as de docentes.

In [97]:
servidores = pd.read_csv(DA_STORAGE_DIR / "transformed/servidores.csv", delimiter=";")

servidores.head()

,nome,matricula,cargo,data_ingresso,atribuicao,carga_horaria
0,Abadio Antonio dos Santos,270698,doscente,1958-04-01,PROFESSOR ENS BASICO TECN TECNOLOGICO,40 HORAS SEMANAIS
1,Abdalla Antonios Kayed Elias,1444504,doscente,2004-02-17,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
2,Abel Ferreira de Lima,6050871,técnico,1985-03-06,MOTORISTA,40 HORAS SEMANAIS
3,Abel Ferreira de Lima,50871,técnico,1985-03-06,MOTORISTA,40 HORAS SEMANAIS
4,Abelson Oliveira de Andrade,50644,técnico,1969-07-01,MOTORISTA,40 HORAS SEMANAIS


In [98]:
servidores.shape

(5435, 6)

In [99]:
docentes = pd.read_csv(DA_STORAGE_DIR / "transformed/docentes.csv", delimiter=";")

docentes.head()

,nome,matricula,disciplina_ministrada,campus
0,Abdalla Antonios Kayed Elias,1444504,Física,CÂMPUS GOIÂNIA
1,Abilio de Jesus Carrascal,1526346,Artes,CÂMPUS ÁGUAS LINDAS
2,Adel Fernando de Almeida Vanny,1791027,Filosofia,CÂMPUS INHUMAS
3,Aderito Schneider Alencar e Tavora,3058944,Não Informada,CÂMPUS CIDADE DE GOIÁS
4,Adria Assuncao Santos de Paula,2864372,Psicologia,CÂMPUS GOIÂNIA OESTE


In [100]:
docentes.shape

(1117, 4)

## Junção dos dois datasets

Para combinar os dois datasets a coluna de matrícula foi utilizada como critério de junção, já que a sua informação é compartilhada pelo dois datasets.

In [101]:
# Realiza um join se baseando na coluna matrícula dos dois dataframes.
# No dataset de servidores é feito um recorte retirando a coluna nome.
docentes_merged = docentes.merge(servidores.iloc[:, 1:], on="matricula")

docentes_merged.head()

,nome,matricula,disciplina_ministrada,campus,cargo,data_ingresso,atribuicao,carga_horaria
0,Abdalla Antonios Kayed Elias,1444504,Física,CÂMPUS GOIÂNIA,doscente,2004-02-17,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
1,Abilio de Jesus Carrascal,1526346,Artes,CÂMPUS ÁGUAS LINDAS,doscente,2014-05-07,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
2,Adel Fernando de Almeida Vanny,1791027,Filosofia,CÂMPUS INHUMAS,doscente,2010-06-15,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
3,Aderito Schneider Alencar e Tavora,3058944,Não Informada,CÂMPUS CIDADE DE GOIÁS,doscente,2018-07-31,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
4,Adria Assuncao Santos de Paula,2864372,Psicologia,CÂMPUS GOIÂNIA OESTE,doscente,2013-08-01,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA


In [102]:
docentes_merged.shape

(1117, 8)

In [103]:
docentes_merged["carga_horaria"].unique()

array(['DEDICACAO EXCLUSIVA', '20 HORAS SEMANAIS', '40 HORAS SEMANAIS'],
      dtype=object)

In [104]:
docentes_merged.groupby(["campus"])["campus"].count()

campus
CÂMPUS ANÁPOLIS                 75
CÂMPUS APARECIDA DE GOIÂNIA     73
CÂMPUS CIDADE DE GOIÁS          48
CÂMPUS FORMOSA                  65
CÂMPUS GOIÂNIA                 341
CÂMPUS GOIÂNIA OESTE            59
CÂMPUS INHUMAS                  67
CÂMPUS ITUMBIARA                63
CÂMPUS JATAÍ                    87
CÂMPUS LUZIÂNIA                 57
CÂMPUS SENADOR CANEDO           40
CÂMPUS URUACU                   62
CÂMPUS VALPARAISO               38
CÂMPUS ÁGUAS LINDAS             42
Name: campus, dtype: int64

In [105]:
# Remove a coluna "cargo", pois já sabemos que todos no dataset são docentes
docentes_merged.drop("cargo", axis=1, inplace=True)

docentes_merged.head()

,nome,matricula,disciplina_ministrada,campus,data_ingresso,atribuicao,carga_horaria
0,Abdalla Antonios Kayed Elias,1444504,Física,CÂMPUS GOIÂNIA,2004-02-17,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
1,Abilio de Jesus Carrascal,1526346,Artes,CÂMPUS ÁGUAS LINDAS,2014-05-07,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
2,Adel Fernando de Almeida Vanny,1791027,Filosofia,CÂMPUS INHUMAS,2010-06-15,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
3,Aderito Schneider Alencar e Tavora,3058944,Não Informada,CÂMPUS CIDADE DE GOIÁS,2018-07-31,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
4,Adria Assuncao Santos de Paula,2864372,Psicologia,CÂMPUS GOIÂNIA OESTE,2013-08-01,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA


## Exportação do novo dataset gerado após o pré-processamento

In [106]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    docentes_merged.to_csv(export_path / "docentes.csv", sep=";", index=False)
except Exception as e:
    print(str(e))